# Classes and Object-Oriented Programming

## Defining classes

A class in Python is effectively a data type. You define a class with the `class` statement.

By convention, class identifiers are in CapCase as in `MyAwesomeClass`.

In [ ]:
class MyAwesomeClass:
    pass

Class instances can be used as structs or records. However, unlike C structures or Java classes, the data fields of an instance don't need to be declared ahead of time.

The following snippet declares an *empty* `Circle` class, and then after initialization, it defines a new data field `radius` and use it in a calculation:

In [5]:
from math import pi

class Circle:
    ...

my_circle = Circle()
my_circle.radius = 5
print(f"The length of the circle is {2 * pi * my_circle.radius:.3f}")

The length of the circle is 31.416


You can initialize the fields of a class instance automatically by including an `__init__()` method. This function is run every time an instance of the class is created.

Python classes may only have one `__init__()` method, and its first argument is set to the newly created class instance when `__init__()` is run.

By convention, `self` is always the name of this first argument.

| NOTE: |
| :---- |
| The `__init__()` method is similar to a constructor, but it doesn't construct anything &mdash; it just initializes the fields of the class. Python has something that is more of a constructor: the `__new__()` method that is called on object creation and returns an uninitialized object. It's rarely used in Python. |

In [11]:
class Circle:
    def __init__(self):
        self.radius = 1

my_circle = Circle()
print(f"Length of the circle: {2 * pi * my_circle.radius:.3f}")

my_circle.radius = 5
print(f"Length of the circle: {2 * pi * my_circle.radius:.3f}")

Length of the circle: 6.283
Length of the circle: 31.416


## Instance variables

An instance variable is a variable associated to an instance of a given class. Each class has itws own copy.

In Python, you create an instance variable on-the-fly by assigning to a field of a class instance:

```python
instance.variable = value
```

Accessing an instance variable requires explicit mention of the containing instance (i.e., `instance.variable`).

The following snippet defines a `Rectangle` class:

In [1]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __repr__(self):
        """Dev-level representation of the instance."""
        return f"Rectangle(width={self.width}, height={self.height})"

my_rectangle = Rectangle(4, 2)
my_rectangle

Rectangle(width=4, height=2)

## Methods

A method is a function associated with a particular class.

In [5]:
import math

class Circle:
    def __init__(self, radius=1):
        self.radius = radius

    def area(self):
        return self.radius * self.radius * math.pi

c = Circle()
print(f"area of a circle of radius {c.radius}: {c.area():.3f}")

c = Circle(3)
print(f"area of a circle of radius {c.radius}: {c.area():.3f}")

c.radius = 5
print(f"area of a circle of radius {c.radius}: {c.area():.3f}")

area of a circle of radius 1: 3.142
area of a circle of radius 3: 28.274
area of a circle of radius 5: 78.540


Invoking a method requires using the instance, as in `c.area()`, and this syntax is called a *bound* method invocation.

The first argument of any method is the instance it was invoked by or on, named `self` by convention. Note that in other languages, the argument is implicit, but not in Python.

Python also supports an alternative syntax  called *unbound* method which can be used to invoke a method through the containing class. It is rarely used:

In [ ]:
c = Circle(3)
Circle.area(c)

28.274333882308138

Python transforms a method invocation `instance.method(arg1, arg2, ...)` into normal function calls using the following rules:

1. Look for the method name in the instance namespace. If a method has been changed or added for this instance, it's invoked in preference over methods in the class or superclass.

2. If the method isn't found in the instance namespace, look up the class type `class` or `instance`, and look for the method there.

3. If the method isn't found, look for the method up the class hierarchy chain.

4. When the method has been found, make a direct call to it as a normal Python function, using the instance as the first argument of the function, and shifting all the other arguments in the method invocation one space over to the right.

Let's see the first and second point in action:

In [12]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def say_hi(self):
        return "Hello from the Rectangle class"

r = Rectangle(4, 2)

def fn_say_hi(self):
    return f"Hello, from a rectangle of width {self.width} and height {self.height}"

# Add the method to the instance
r.say_hi = fn_say_hi.__get__(r, Rectangle)

print(r.say_hi())

r2 = Rectangle(2, 4)
print(r2.say_hi())

Hello, from a rectangle of width 4 and height 2
Hello from the Rectangle class


## Class variables

A class variable is a variable associated with a class, not an instance of a class, and is accessible by all instances of the class.

In [15]:
class Circle:
    pi = 3.14159

    def __init__(self, radius=1):
        self.radius = radius

    def area(self):
        return self.radius * self.radius * Circle.pi

c = Circle()
print(f"{c.area()=}")

print(f"{Circle.pi}")

c.area()=3.14159
3.14159


Hardcoding the name of the class inside the class's methods is not very pretty. This can be avoided through the use of the special `__class__` attribute:

In [16]:
class Circle:
    pi = 3.14159

    def __init__(self, radius=1):
        self.radius = radius

    def area(self):
        return self.radius * self.radius * self.__class__.pi

c = Circle()
print(f"{c.area()=}")

print(f"{Circle.pi}")

c.area()=3.14159
3.14159


That trick is also available outside the class, but it makes the expression less readable:

In [17]:
class Circle:
    pi = 3.14159

    def __init__(self, radius=1):
        self.radius = radius

    def area(self):
        return self.radius * self.radius * self.__class__.pi

c = Circle()

print(c.__class__.pi)

3.14159


It's important to note that there are a few caveats in Python with respect to class variables.

First, in Python, class variables can also be accessed through instances. If an instance has both a class variable and an instance variable with the same name, the instance variable takes precedence.

In [ ]:
class MyClass:
    msg = "foo"

    def __init__(self):
        self.msg = "bar"

class MyOtherClass:
    msg = "foobar"

obj = MyClass()
obj2 = MyOtherClass()

assert obj.msg == "bar"
assert MyClass.msg == "foo"
assert MyOtherClass.msg == "foobar"
assert obj2.msg == "foobar"

Another related caveat, is that because you can do `instance.class_variable` you might expect that doing `instance.class_variable=value` will change the value of the all class variable, but that's not the case:

In [20]:
class MyClass:
    num_instances = 0

    def __init__(self):
        self.__class__.num_instances += 1

assert MyClass.num_instances == 0

obj = MyClass()
assert MyClass.num_instances == 1

obj.num_instances = 5
assert MyClass.num_instances == 1
assert obj.num_instances == 5

## Static Methods and Class Methods

Python allows the definition of two types of methods that are not bound to instances: static methods and class methods.

### Static methods

Static methods are used to define methods that are not bound to any particular instance of the class. They are defined using the `@staticmethod` decorator:

In [22]:
class Circle:
    """Circle class."""
    all_circles = []
    pi = 3.14159

    def __init__(self, radius=1):
        """Create a circle of the given radius."""
        self.radius = radius
        self.__class__.all_circles.append(self)

    def area(self):
        """Determine the area of the Circle."""
        return self.radius * self.radius * self.__class__.pi

    @staticmethod
    def total_area():
        """Static method returning the total area of all Circles."""
        total = 0
        for circle in Circle.all_circles:
            total += circle.area()
        return total

c1 = Circle(1)
c2 = Circle(2)

assert Circle.total_area() == c1.area() + c2.area()

Note that you can access the documentation as it were class and instance variables:

In [23]:
Circle.__doc__

'Circle class.'

In [24]:
c1.area.__doc__

'Determine the area of the Circle.'

### Class methods

Class methods are similar to static methods (in the sense that they're not bound to any class instance), but they are implicitly passed the class they belong to. They are defined using the `@classmethod` decorator.

In [25]:
class Circle:
    """Circle class."""
    all_circles = []
    pi = 3.14159

    def __init__(self, radius=1):
        """Create a circle of the given radius."""
        self.radius = radius
        self.__class__.all_circles.append(self)

    def area(self):
        """Determine the area of the Circle."""
        return self.radius * self.radius * self.__class__.pi

    @classmethod
    def total_area(cls):
        """Static method returning the total area of all Circles."""
        total = 0
        for circle in cls.all_circles:
            total += circle.area()
        return total

c1 = Circle(1)
c2 = Circle(2)

assert Circle.total_area() == c1.area() + c2.area()

As such, if you need to refer to the enclosing class in a static/class method, it's better to use class methods as you won't have to hardcode the class's name.

## Inheritance

The following snippet illustrates what the inheritance syntax looks like in Python:

In [1]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Square(Shape):
    def __init__(self, side=1, x=0, y=0):
        super().__init__(x, y)
        self.side = side

class Circle(Shape):
    def __init__(self, radius=1, x=0, y=0):
        super().__init__(x, y)
        self.radius = radius

Note that the class you inherit from is identified within parentheses in the subclass definition.

Not also that it is required to invoke the `__init__` method of the superclass explicitly (Python won't do that for you).

To do so, you can either use `super().__init__(x, y)` as above, or hardcode the class name (as in `Shape.__init__(self, x, y)`). The first option might bring some problems in complex cases, while the second has the drawback of hardcoding the class' name, which will create problems if the class hierarchy changes.

In Python, all instances of the subclasses can make use of the methods defined in the superclass (i.e., all Python methods are *virtual*):

In [3]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def move(self, delta_x, delta_y):
        self.x += delta_x
        self.y += delta_y

class Square(Shape):
    def __init__(self, side=1, x=0, y=0):
        super().__init__(x, y)
        self.side = side

class Circle(Shape):
    def __init__(self, radius=1, x=0, y=0):
        super().__init__(x, y)
        self.radius = radius

c = Circle(1)
assert c.x == 0
assert c.y == 0

c.move(3, 4)

assert c.x == 3
assert c.y == 4

f"{c.x=}, {c.y=}"

'c.x=3, c.y=4'

The following snippet introduces a `Rectangle` class and adjusts the hierarchy, so that `Square` will inherit from `Rectangle` instead of from `Shape`. 

In [ ]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def move(self, delta_x, delta_y):
        self.x += delta_x
        self.y += delta_y

class Rectangle(Shape):
    def __init__(self, width, height, x=0, y=0):
        super().__init__(x, y)
        self.width = width
        self.height = height

class Square(Rectangle):
    def __init__(self, side=1, x=0, y=0):
        super().__init__(side, side, x, y)
        self.side = side

class Circle(Shape):
    def __init__(self, radius=1, x=0, y=0):
        super().__init__(x, y)
        self.radius = radius

my_square = Square(1)

## Inheritance with class and instance variables

Consider the following snippet showing a simple class hierarchy, where P is the parent class of C. We have methods that set the value of an instance variable `x` defined both in the parent and subclass.

In [6]:
class P:
    z = "Hello"
    def set_p(self):
        self.x = "Class P"
    def print_p(self):
        print(self.x)

class C(P):
    def set_c(self):
        self.x = "Class C"
    def print_c(self):
        print(self.x)

c = C()
c.set_p()       # set x = "Class P" in the subclass
c.print_p()     # print 'Class P'

c.print_c()     # print 'Class P'
c.set_c()       # set x = "Class C" in the subclass
c.print_c()     # print 'Class C'
c.print_c()     # print 'Class C'

Class P
Class P
Class C
Class C


That is, only one instance variable of a given name exists for a given instance, regardless the class hierarchy.

That is, this is true regardless which class defines the method being invoked on the instance.

Instance variables defined in a superclass are inherited in the subclass, as seen on the Shape hierarchy:

In [7]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Rectangle(Shape):
    def __init__(self, width, height):
        super().__init__(0, 0)
        self.width = width
        self.height = height

r = Rectangle(4, 2)
assert r.x == 0
assert r.y == 0

Class variables are inherited, but defining the same name for a class variable in a superclass and a subclass is discouraged, because it can lead to confusion when changing its value, as seen in [Class Variables](#class-variables).

In [8]:
class P:
    z = "Hello"
    def set_p(self):
        self.x = "Class P"
    def print_p(self):
        print(self.x)

class C(P):
    def set_c(self):
        self.x = "Class C"
    def print_c(self):
        print(self.x)

c = C()

assert c.z == "Hello"  # access to class variable looks like instance variable
assert P.z == "Hello"  # Good
assert C.z == "Hello"  # Confusing, because it is inherited

As seen in [Class Variables](#class-variables) changing the value of a class variable can lead to somewhat unexpected results:

In [ ]:
C.z = "Bonjour"

assert c.z == "Bonjour"  # Accessing class variable through the instance
assert C.z == "Bonjour"  # Accessing class variable through the subclass
assert P.z == "Hello"    # The parent class class variable hasn't changed

And it's much worse, if we try to change the value of the class variable through the instance, because a new instance variable will be created.

In [12]:
class P:
    z = "Hello"
    def set_p(self):
        self.x = "Class P"
    def print_p(self):
        print(self.x)

class C(P):
    def set_c(self):
        self.x = "Class C"
    def print_c(self):
        print(self.x)

c = C()

c.z = "Bonjour"

assert c.z == "Bonjour"  # Accessing class variable through the instance
assert C.z == "Hello"    # Accessing class variable through the subclass
assert P.z == "Hello"    # The parent class class variable hasn't changed

### Exercise

1. Create a base class `Shape` that takes the coordinates of a point `(x, y)` as two individual arguments. Define a method `move(delta_x, delta_y)` that performs the translation of the shape's point.

2. Create a `Circle` class that inherits from `Shape`. Include two class variables `pi=3.14.159` and `all_circles`, a list that contains a reference to each of the instances created. The initializer method must optionally accept the radius (default value = 1), and the coordinates x, y, which should be passed to the base class initializer. In the class, define a class method `total_area` which returns the area of all the circles defined. Define also a static method `circle_area` returning the area of a circle of radius `r`.

3. Create an instance of `Circle` named `c1` by invoking the default initializer and print the radius, and (x, y) values. Then create another instance `c2` with radius 2, and (x, y) = (1, 1) and print the radius and (x, y) values. Call the `move()` method on `c2` with delta_x = 2 and delta_y = 2. Print, the radius and (x, y) again. Print `Circle.all_circles` and check its contents. Invoke `total_area()`. Invove the static method `circle_area` for the radius of circle c1.

In [14]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def move(self, delta_x, delta_y):
        self.x += delta_x
        self.y += delta_y

class Circle(Shape):
    pi = 3.14159
    all_circles = []  # noqa: RUF012

    def __init__(self, radius = 1, x = 0, y = 0):
        super().__init__(x, y)
        self.radius = radius
        self.__class__.all_circles.append(self)

    @classmethod
    def total_area(cls):
        result = 0
        for circle in cls.all_circles:
            result += cls.pi * circle.radius * circle.radius
        return result

    @staticmethod
    def circle_area(radius):
        return Circle.pi * radius * radius

c1 = Circle()
print(f"Default circle: {c1.radius=}, {c1.x=}, {c1.y=}")

c2 = Circle(2, 1, 1)
print(f"Custom init circle: {c2.radius=}, {c2.x=}, {c2.y=}")

c2.move(2, 2)
print(f"c2: {c2.radius=}, {c2.x=}, {c2.y=}")

print(f"{Circle.all_circles=}")
assert len(Circle.all_circles) == 2  # noqa: PLR2004
assert Circle.all_circles == [c1, c2]

print(f"{Circle.total_area()=}")
print(f"{Circle.circle_area(c1.radius)=}")


Default circle: c1.radius=1, c1.x=0, c1.y=0
Custom init circle: c2.radius=2, c2.x=1, c2.y=1
c2: c2.radius=2, c2.x=3, c2.y=3
Circle.all_circles=[<__main__.Circle object at 0x7fed5e65b290>, <__main__.Circle object at 0x7fed5e658770>]
Circle.total_area()=15.70795
Circle.circle_area(c1.radius)=3.14159


## Private variables and private methods

A private variable or private method is one that can't be seen outside of the class in which it's defined.

A class may define a private variable and inherit from a class that defines a private variable of the same name, but this doesn't cause a problem, because the fact that the variables are private ensures that separate copies of them are kept.

In Python, private variables and methods are identified by prefixing them with a double underscore.

In [18]:
class SomeClass:
    def __init__(self):
        self.x = 2
        self.__y = 3

    def print_y(self):
        print(self.__y)

obj = SomeClass()
print(obj.x)
try:
    print(obj.__y)  # noqa: SLF001
except AttributeError as e:
    print(f"Oops: {e}")

obj.print_y()

2
Oops: 'SomeClass' object has no attribute '__y'
3


The way in which Python makes variables and methods private is by mangling its name:

In [19]:
dir(obj)

['_SomeClass__y',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'print_y',
 'x']

In [ ]:
# Accessing the private instance variable directly
print(obj._SomeClass__y)  # noqa: SLF001


3


This means that the purpose of prefixing the variables/methods is prevent accidental access &mdash; if someone wanted to, he could deliberately access the mangled library.

## Using `@property` for more flexible instance variables

Python allows direct access to instance variables directly. While this need for getters and setters makes Python classes cleaner, there are situations in which controlling access to instance variables might come in handy.

Python provides the `@property` decorator to do so. The name of the decorated method will become the property's name, and the method will become the getter of a read-only property.

In [ ]:
class Temperature:
    def __init__(self):
        self._temp_fahr = 0
    @property
    def temp(self):
        return (self._temp_fahr - 32) * 5 / 9

The setter is defined using the `@<property>.setter` decorator:

In [22]:
class Temperature:
    def __init__(self):
        self._temp_fahr = 0

    @property
    def temp(self):
        return (self._temp_fahr - 32) * 5 / 9

    @temp.setter
    def temp(self, new_temp):  # noqa: ANN202
        self._temp_fahr = new_temp * 9 / 5 + 32

t = Temperature()
print(f"{t._temp_fahr=}")  # noqa: SLF001

print(f"{t.temp=}")

t._temp_fahr=0
t.temp=-17.77777777777778


### Exercise

Create a `Rectangle` class in which the properties for `width` and `height` are private, but those are exposed with `setters` and `getters` that don't allow negative errors.

In [24]:
class Rectangle:
    def __init__(self, width, height):
        self.__class__.ensure_positive_dimension("width", width)
        self.__class__.ensure_positive_dimension("height", height)
        self.__width = width
        self.__height = height

    @property
    def width(self):
        return self.__width

    @width.setter
    def width(self, width):
        self.__class__.ensure_positive_dimension("width", width)
        self.__width = width

    @property
    def height(self):
        return self.__height

    @height.setter
    def height(self, height):
        self.__class__.ensure_positive_dimension("height", height)
        self.__height = height

    @staticmethod
    def ensure_positive_dimension(dimension_str, value):
        if value > 0:
            return
        msg = f"{dimension_str} must be positive"
        raise ValueError(msg)


r = Rectangle(4, 2)
print(f"{r.width=}, {r.height=}")

r.width = 5
r.height = 3
print(f"{r.width=}, {r.height=}")

r.width=4, r.height=2
r.width=5, r.height=3


## Scoping rules and namespaces for class instances

When you're in a method of a class, you have direct access to:
+ the local namespace: parameters and variables defined in the method.
+ the global namespace: functions and variables declared at the module level.
+ the built-in namespace: built-in functions and exceptions.

These are searched in that order: local >> global >> built-in

You also have access through the `self` variable to the:
+ the instance's namespace: instance variables, private instance variables, and superclass instance's variables.
+ the class' namespace: methods, class variables, private methods, and private class variables.
+ the superclass' namespaces: superclass methods and superclass class variables.

These are searched in that order: instance >> class >> superclass.

Private instance variables, private superclass methods 

## Destructors and memory management

Unlike other languages, creating and calling a destructor isn't necessary to ensure that memory used by your instance is freed.

Python provides automatic memory management through a reference-counting mechanism. When the count for a given object reaches zero, the memory used by your instance is reclaimed.

Occasionally, you might need to deallocate an external resource explicitly when an object is removed. In such cases, you should rely on a context manager:

```python
with context_manager() as cm:
    # ... do something with cm ...

# at this point, cm had been deallocated
```

## Multiple inheritance

In Python, a class can inherit from any number of parent classes in the same way that it can inherit from a single parent class. 

If none of the involved classes, including those inherited indirectly through a parent class, contains instance variables or methods of the same name, the inherited class behaves like a synthesis of its own definitions and all of its ancestors' definitions.

In the event that some of the classes share method names, Python searches base classes when looking for a method not defined in the original class on which the method was invoked, in left-to-right order, and looking through all of the ancestor classes of a base class before looking in the next base class.

The recommendation is to stick to the more standard uses of multiple inheritance, as in the creation of mixin or addin classes, to favor readability avoiding name clashes.

### Exercise: class hierarchy to build HTML document

Create a hierarchy of classes that could be used to generate an HTML page just by calling the `__str__()` method so that the following code:

```python
para = p(text="this is some body text wrapped in a paragraph")
doc_body = body(text="This is the body", subelement=para)
doc = html(subelement=doc_body)
print(doc)
```

Should generate::

```html
<html>
    <body>
        This is the body
        <p>
            This is some body text wrapped in a paragraph
        </p>
    </body>
</html>
```

The approach described above is a bit sortsighted. Instead of using a single `subelement` and optionally a text can be enhanced to support any number of children elements. That way, the body, which has both some inner text and a paragraph, can be easily accommodated.

The resulting classes are as follows.

In [ ]:
class HtmlElement:
    def __init__(self, tag: str, children: list["HtmlElement"] | None = None):
        self.tag = tag
        self.children = children

    def __str__(self) -> str:
        """User oriented representation of the object."""
        result_str = f"<{self.tag}>"
        for child in self.children:
            result_str += str(child)
        result_str += f"</{self.tag}>"
        return result_str


class InnerText(HtmlElement):
    def __init__(self, text: str):
        super().__init__(tag=None, children=None)
        self.text = text

    def __str__(self):
        """User oriented representation of the instance."""
        return self.text

class ParaTag(HtmlElement):
    def __init__(self, children):
        super().__init__(tag="p", children = children)

class BodyTag(HtmlElement):
    def __init__(self, children):
        super().__init__(tag="body", children=children)

class HtmlTag(HtmlElement):
    def __init__(self, children):
        super().__init__(tag="html", children=children)

para_inner_text = InnerText("This is some body text wrapped in a paragraph")
para = ParaTag(children=[para_inner_text])

body_inner_text = InnerText("This is the body")
body = BodyTag(children=[body_inner_text, para])

html = HtmlTag(children=[body])

str(html)

'<html><body>This is the body<p>This is some body text wrapped in a paragraph</p></body></html>'